# Get Recipe description from TikTok

## Setup import paths


In [1]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

## Instantiate task object

Enter valid tiktok URL with recipe


In [10]:
url = "https://vm.tiktok.com/ZMDJuuAJj/"

In [11]:
from recipebot.adapters.services.groq_tt_parser.tt_recipe_parser import (
    GroqTTRecipeParser,
)
from recipebot.adapters.services.tt_resolver import HttpxTTResolver
from recipebot.config import settings
from recipebot.infra.groq.client import GroqClient
from recipebot.infra.transport.httpx_transport import init_transport
from recipebot.tasks.recipe_from_tt.recipe_from_tt import RecipeFromTTTask

async with init_transport() as httpx_transport:
    groq_client = GroqClient(settings.GROQ_SETTINGS)

    task = RecipeFromTTTask(
    tt_resolver=HttpxTTResolver(httpx_transport),
    recipe_parser=GroqTTRecipeParser(groq_client),
)
    
    dto = await task.run(url)

print(dto.model_dump_json(indent=2))

{
  "title": "Chinese-style Chicken",
  "ingredients": [
    {
      "name": "chicken",
      "qty": "600",
      "unit": "g",
      "group": "Main"
    },
    {
      "name": "flour",
      "qty": "3",
      "unit": "tbsp",
      "group": "Main"
    },
    {
      "name": "egg",
      "qty": "1",
      "unit": "pc",
      "group": "Main"
    },
    {
      "name": "seasonings",
      "qty": null,
      "unit": null,
      "group": "Main"
    },
    {
      "name": "sugar",
      "qty": "1.5",
      "unit": "tbsp",
      "group": "Sauce"
    },
    {
      "name": "soy sauce",
      "qty": "3",
      "unit": "tbsp",
      "group": "Sauce"
    },
    {
      "name": "ketchup",
      "qty": "3",
      "unit": "tbsp",
      "group": "Sauce"
    },
    {
      "name": "vegetable oil",
      "qty": "1.5",
      "unit": "tbsp",
      "group": "Sauce"
    }
  ],
  "steps": [
    "Cut chicken fillet into small pieces.",
    "In a bowl mix flour, egg and seasonings.",
    "Coat chicken pieces i

## Groq API usage/time



--- API Token Usage --- 
Total Tokens: 1390
Prompt Tokens: 1086
Completion Tokens: 304
Cached Tokens: 0

--- Response Time --- 
Completion Time: 0.829232888 seconds
Queue Time: 0.163276288 seconds
Prompt Time: 0.01392403 seconds
Total Time: 0.843156918 seconds


# Use Calorie API to get nutrients data for ingredients


In [64]:
async def get_nutrition(review: Recipe):  # Assuming you pass the recipe object here
    # 1. Build the query string cleanly
    # Result: "600 g chicken, 3 tbsp flour, 1 pc egg"
    ingredients_list = [i.basic_info() for i in review.ingredients]
    query_string = ", ".join(ingredients_list)

    print(f"Generated Query: {query_string}\n")

    async with init_transport() as transport:
        data = HTTPRequestData(
            url="https://api.calorieninjas.com/v1/nutrition",
            method="GET",
            headers={
                "X-Api-Key": "ozLAbQNtp99NukKo9o5GNA==vAAfjNMO4fElYKBU"
            },  # Don't forget your actual key
            # 2. FIX: Pass the actual variable 'query_string', not hardcoded "chicken"
            params={"query": query_string},
        )

        response_json = await transport.request(data)

        # Parse the JSON string into a dict
        print(response_json)

        # Optional: Print a pretty summary
        total_cal = 0
        print("\n\n\n--- Nutrition Facts ---")
        for item in response_json.get("items", []):
            print(f"{item['name']}: {item['calories']} kcal")
            total_cal += item["calories"]
        print(f"Total: {total_cal} kcal")


await get_nutrition(review)

Generated Query: 600 g chicken, 3 tbsp flour, 1 piece egg, seasonings, 1.5 tbsp sugar, 3 tbsp soy sauce, 3 tbsp ketchup, 1.5 tbsp vegetable oil

{'items': [{'name': 'chicken', 'calories': 1335.8, 'serving_size_g': 600.0, 'fat_total_g': 77.4, 'fat_saturated_g': 21.9, 'protein_g': 142.4, 'sodium_mg': 433, 'potassium_mg': 1074, 'cholesterol_mg': 552, 'carbohydrates_total_g': 0.3, 'fiber_g': 0.0, 'sugar_g': 0.0}, {'name': 'flour', 'calories': 1359.7, 'serving_size_g': 375.0, 'fat_total_g': 3.7, 'fat_saturated_g': 0.6, 'protein_g': 38.4, 'sodium_mg': 7, 'potassium_mg': 399, 'cholesterol_mg': 0, 'carbohydrates_total_g': 288.6, 'fiber_g': 10.1, 'sugar_g': 1.0}, {'name': 'egg', 'calories': 73.5, 'serving_size_g': 50.0, 'fat_total_g': 4.8, 'fat_saturated_g': 1.6, 'protein_g': 6.3, 'sodium_mg': 69, 'potassium_mg': 99, 'cholesterol_mg': 185, 'carbohydrates_total_g': 0.4, 'fiber_g': 0.0, 'sugar_g': 0.2}, {'name': 'sugar', 'calories': 242.9, 'serving_size_g': 63.0, 'fat_total_g': 0.0, 'fat_saturate